In [1]:
import sklearn
import cv2 as cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from util import draw_bboxes


In [2]:
def bgr_to_rgb(image):
    b,g,r = cv2.split(image)
    return cv2.merge([r,g,b])
def draw_rect(img, box):
    x, y, w, h = box
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 2)
def get_coord(box):
    x1, y1, w, h = box
    return x1, y1, x1 + w, y1 + h
def get_area(box):
    return box[2] * box[3]
def get_center(box):
    return box[0] + box[2] / 2, box[1] + box[3] / 2
def get_center_velocity(track_info):
    return track_info[4], track_info[5]
def add_velocity(center, velocity):
    return center[0] + velocity[0], center[1] + velocity[1]
def l2_dist_square(p1, p2):
    return (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2 
def check_overlap(box, proposal):
    print(box)
    print(proposal)
    b_x1, b_y1, b_x2, b_y2 = get_coord(box)
    p_x1, p_y1, p_x2, p_y2 = get_coord(proposal)
    print((b_x1, b_y1, b_x2, b_y2))
    print((p_x1, p_y1, p_x2, p_y2))
    i_x1 = max(b_x1, p_x1)
    i_x2 = min(b_x2, p_x2)
    i_y1 = max(b_y1, p_y1)
    i_y2 = min(b_y2, p_y2)
    
    print(i_x1)
    print(i_x2)
    print(i_y1)
    print(i_y2)
    if i_x1 >= i_x2 or i_y1 >= i_y2:
        # Do not interset
        return 0
    elif p_x1 >= b_x1 and p_x2 <= b_x2 and p_y1 >= b_y1 and p_y2 <= b_y2:
        # Contained inside the box
        return 1
    else:
        # Enter / exit
        return 2
def check_entering(box, proposal, track_info):
    box_center = get_center(box)
    proposal_center = get_center(proposal)
    proposal_velocity = get_center_velocity(track_info)
    # track info [u, v, r, h, v_x, v_y, v_r, v_h]  
    old_dist = l2_dist_square(box_center, proposal_center)
    new_dist = l2_dist_square(box_center, add_velocity(proposal_center, proposal_velocity))
    if new_dist < old_dist:
        return True
    else:
        return False
    
def tracking_box(img_seq, proposals_seq, instances_seq, tracking_seq, box):
    # box: [x, y, w, h]
    output_seq = []
    for i in range(len(img_seq)):
        img = img_seq[i].copy()
        proposals_list = proposals_seq[i]
        instances_list = instances_seq[i]
        tracking_list = tracking_seq[i]
        
        n_pedestrians_inside = 0
        n_pedestrians_enter = 0
        n_pedestrians_leave = 0
        
        draw_rect(img, box)
        img = draw_bboxes(img, proposals_list, instances_list)
        for j in range(len(proposals_list)):
            proposal = proposals_list[j]
            instance = instances_list[j]
            track_info = tracking_list[j]
            
            mode = check_overlap(box, proposal)
            print("mode = " + str(mode))
            if mode == 0:
                # Do not interset
                pass
            elif mode == 1:
                # Contained inside the box
                n_pedestrians_inside += 1
            else:
                # mode == 2:
                # velocity checking
                if check_entering(box, proposal, track_info):
                    n_pedestrians_enter += 1
                else:
                    n_pedestrians_leave += 1
        
        text = "Inside: " + str(n_pedestrians_inside) + "  Entering: " + str(n_pedestrians_enter) + "  Leaving: " + str(n_pedestrians_leave)
        cv2.putText(img, text, (box[0], box[1]), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        output_seq.append(img)
        
    return output_seq

In [3]:
x = 100
y = 100
w = 300
h = 200


box = [x, y, w, h]


In [4]:
img1 = cv2.imread('./sequence/000001.jpg')
img2 = cv2.imread('./sequence/000002.jpg')
img3 = cv2.imread('./sequence/000003.jpg')
img_seq = [img1, img2, img3]
img_seq = [img1]

In [6]:
proposals_seq = [[[50, 75, 100, 50], [300, 300, 50, 50]]]

tracking_seq = [[[100, 100, 1, 1, -10, -5, 0, 0], [100, 75, 1, 1, 10, 5, 0, 0]]]
instances_seq = [[1, 2]]
output_seq = tracking_box(img_seq, proposals_seq, instances_seq, tracking_seq, box)

AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
plt.imshow(bgr_to_rgb(output_seq[0]))